In [61]:
from openai import OpenAI
import os
import requests
import json

open_api_key = os.getenv('OPENAI_API_KEY')
translate_key = os.getenv('translate_key')

# Python에서는 client 불러와서 api key 입력하고 api 날림.
client = OpenAI(
    api_key= open_api_key,
)

# audio의 information 반환 audios_ids를 input으로 주면 지금 status랑 url 주소가 반환됨. 실시간 업데이트됨.
# data[0]["status"] 상태
# data[0]['audio_url'] 1번 노래의 주소
# data[1]['audio_url'] 1번 노래의 주소
def get_audio_information(base_url,audio_ids):
    url = f"{base_url}/api/get?ids={audio_ids}"
    response = requests.get(url)
    return response.json()

# 노래의 제목을 지어주는 gpt 코드 가사,스타일(한글)을 입력값으로 받음 get_audio_information을 통해 조회 가능.
def generate_title(lyrics, style):
    
    prompt = f"""
    너는 노래 제목을 지을 거야. 사용자가 제공한 노래 가사와 스타일을 기반으로 적합한 제목을 만들어줘.

    노래의 스타일: {style}
    
    노래 가사:
    {lyrics}

    예시:
    스타일: 발라드
    가사:
    어느 날 거울 앞에 서서 봤어
    내 모습이 마음에 들지 않아
    언제부터인가 난 힘없이 웃음 짓고
    꽤나 힘든 하루를 보낼 때가 많았어

    매일 변하는 유행에 쫓기며
    바쁘게 웃으면서도 가면을 써
    하지만 진정한 나를 찾아가는 여정에서
    내 안의 꽃들이 피기 시작했어

    내가 꽤나 예쁜 여자가 되어가는 중
    매일 조금씩 변해가는 거잖아
    내 안의 빛을 찾아 걸어가는 중
    섬세한 나만의 모습을 발견하고 있어

    제목: 내 안의 빛

    이 정보를 바탕으로, 주어진 노래 가사와 스타일에 맞는 적합한 한국어 노래 제목을 10글자 미만으로 지어줘.
    
    """
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=500,
    temperature=0
    )
    
    title = response.choices[0].message.content.strip()
    
    if "제목:" in title:
        return title[3:]
    else:
        return title

# 노래를 만드는 api 코드 url,가사,스타일(영어),제목(영어)를 입력값으로 받음       
# response.json() 중 id값을 통해 
def create_music(base_url,lyrcis,tags,title):
    
    url = f"{base_url}/api/custom_generate"
    
    payload = {
        "prompt": f"{lyrcis}",
        "tags": f"{tags}",
        "title": f"{title}",
        "make_instrumental": False,
        "model": "chirp-v3-5",
        "wait_audio": False
    }

    headers = {'Content-Type': 'application/json'}
    response = requests.post(url, data=json.dumps(payload), headers=headers)
    
    return response.json()

# 표지 만드는 ai ,스타일(한글),제목(한글),가사(한글) 입력 받음.
# image의 url 리턴함.
def create_sign(style,tilte,lyrics):
    
    prompt = f"""
    Create an album cover for a {style} song. Reflect the lyrics: '{lyrics}'.
    The cover should visually represent the mood and theme of the song. Do not include any text.
    """


    client = OpenAI(api_key= api_key,)

    response = client.images.generate(
      model="dall-e-3",
      prompt= prompt,
      size="1024x1024",
      quality="standard",
      n=1,
    )

    return response.data[0].url

# 번역 API 한국어를 영어로 번역.
def Translate(text):

    params = {'auth_key' : '40157f7d-ffa8-482a-8029-4eb0fd4201fe:fx', 'text' : text, 'source_lang' : "KO", "target_lang": "EN" }
    result = requests.post('https://api-free.deepl.com/v2/translate', data=params, verify=False)
    return result.json()['translations'][0]["text"]

# 가사 써주는 AI 노래 스타일(한국) 입력 받으면 가사 리턴.
def generate_lyrics(style):

    prompt = f"""
    너는 노래 가사를 작성할 거야. 노래의 장르와 스타일은 다음과 같아:

    장르와 스타일: {style}

    예시:
    Verse 1:
    어느 날 거울 앞에 서서 봤어
    내 모습이 마음에 들지 않아
    언제부터인가 난 힘없이 웃음 짓고
    꽤나 힘든 하루를 보낼 때가 많았어

    Verse 2:
    매일 변하는 유행에 쫓기며
    바쁘게 웃으면서도 가면을 써
    하지만 진정한 나를 찾아가는 여정에서
    내 안의 꽃들이 피기 시작했어

    Chorus:
    내가 꽤나 예쁜 여자가 되어가는 중
    매일 조금씩 변해가는 거잖아
    내 안의 빛을 찾아 걸어가는 중
    섬세한 나만의 모습을 발견하고 있어

    Verse 1:
    사람들의 시선이 어지러울 때
    내 안의 소리에 귀 기울이며
    지금껏 보여줄 수 없었던 나의 자신을
    천천히 발견해 나가는 중인 거야

    Verse 2:
    너무 급하게 변하려 하지 않아
    차분히 성장하는 여자로
    마침내 믿음을 지녔을 때
    진짜로 내가 원하는 나로 다가갈 거야

    Chorus:
    내가 꽤나 예쁜 여자가 되어가는 중
    매일 조금씩 변해가는 거잖아
    내 안의 빛을 찾아 걸어가는 중
    섬세한 나만의 모습을 발견하고 있어

    Bridge:
    가끔은 힘들 수 있으나 포기하지 않을 거야
    내가 원하는 그 순간을 위해

    Chorus:
    내가 꽤나 예쁜 여자가 되어가는 중
    매일 조금씩 변해가는 거잖아
    내 안의 빛을 찾아 걸어가는 중
    섬세한 나만의 모습을 발견하고 있어
    내 마음 모두 너에게 줄게

    이 정보를 바탕으로, {style} 스타일에 맞는 한국어 노래 가사를 작성해줘. 장르에 유의해줘. 노래 가사는 다음과 같은 구조를 가질 수 있어.:
    
    노래가 락또는 팝이면:
    - (Verse 1)
    - (Chorus)
    - (Verse 2)
    - (Chorus)
    - (Bridge) (선택 사항)
    - (Chorus)
    
    노래가 트로트이면:
    -(Intro)
    -(Verse 1)
    -(Chorus)
    -(Verse 2)
    -(Chorus)
    -(Chorus) 
    -(Outro) 

    노래가 발라드이면:
    -(Verse 1)
    -(Chorus)
    -(Verse 2)
    -(Chorus)
    -(Verse 3)
    -(Chorus)
    -(Bridge)
    -(Chorus)
    -(Outro)

    노래가 힙합이면:
    - (Verse 1)
    - (Chorus)
    - (Verse 2)
    - (Chorus)
    - (Verse 3)
    - (Chorus)

    가사는 최소한 150자 이상으로 작성되어야 해.
    """


    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        temperature=0
    )
    
    return response.choices[0].message.content.strip()

# 사용자가 입력한 자연어로된 스타일을 키워드로 뽑아줌.
def extract_style(origin_style):

    prompt = f"""
    The user has provided a text describing a song style. Extract the main keywords from this text that summarize its core elements.

    Example:
    Text: 트로트 남자로 해줘 나는 활기찬 사랑 노래 만들고 싶어요.
    Keywords: 남자,트로트,활기찬,사랑

    Text:
    {origin_style}

    Keywords:
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
        max_tokens=500
    )
    #print(style_keyword)
    style_keyword = response.choices[0].message.content.strip()
    
    if "Keywords:" in style_keyword:
        return style_keyword[9:]
    
    return style_keyword

In [5]:
genre = "트로트로 해줘" #장르 입력받기
sex = "여자로 해줘" # 성별 입력받기
origin_style = "흥겨운 노래" # 느낌 입력 받기
 
# 인풋값 스타일로 합치기.
styles = genre + sex + origin_style

style_keyword = extract_style(styles) #스타일 키워드화
print(style_keyword)

lyrics = generate_lyrics(style_keyword) # 가사 추출
title  = generate_title(lyrics,style_keyword) # 제목 추출
print(title)

 트로트, 여자, 흥겨운


In [13]:
# 노래 퀄리티 향상을 위한 예외처리
en_style_keyword = "Korean," + Translate(style_keyword) + ",Good Quality"

# trot는 정해진 포맷으로 밖에 처리가 안됨.
if "trot" in en_style_keyword:
    en_style_keyword = "trot, Korean trot"

# vocal로 치환해주기.
if "woman" in en_style_keyword:
    en_style_keyword += ",Female vocal"
else:
    en_style_keyword += ",Male vocal"
    
print(en_style_keyword)
en_title = Translate(title)
print(en_title)

/home/tester01/anaconda3/envs/mintai_llm/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api-free.deepl.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


trot, Korean trot,Male vocal


/home/tester01/anaconda3/envs/mintai_llm/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api-free.deepl.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


"The story of a happy smile"


In [14]:
music_response = create_music("http://localhost:3000",lyrics,en_style_keyword,en_title)
ids = f"{music_response[0]['id']},{music_response[1]['id']}" # music_reponse에서 음악들의 id들 추출.

In [50]:
response = create_sign(style_keyword,title,lyrics)

In [15]:
# 5초마다 갱신하면서 data 상태 확인 참고코드.
import time

streaming_switch = 1
complete_switch = 1
t = 0
for _ in range(60):
    data = get_audio_information("http://localhost:3000",ids)
    if data[0]["status"] == 'streaming' and streaming_switch:
        print(f"{data[0]['id']} ==> {data[0]['audio_url']}")
        print(f"{data[1]['id']} ==> {data[1]['audio_url']}")
        streaming_switch = 0
    if data[0]["status"] != 'streaming' and complete_switch:
        print(f"{data[0]['id']} ==> {data[0]['audio_url']}")
        print(f"{data[1]['id']} ==> {data[1]['audio_url']}")
        complete_switch = 0
        
    time.sleep(5)
    t += 5
    print(t)

f31e472a-c90f-4ebe-97c7-fb34ef7baae3 ==> 
5392c83e-d629-4ab0-a9e4-51544d3d6411 ==> 
5
f31e472a-c90f-4ebe-97c7-fb34ef7baae3 ==> https://audiopipe.suno.ai/?item_id=f31e472a-c90f-4ebe-97c7-fb34ef7baae3
5392c83e-d629-4ab0-a9e4-51544d3d6411 ==> https://audiopipe.suno.ai/?item_id=5392c83e-d629-4ab0-a9e4-51544d3d6411
10
15
20
25
30
35
40
45


KeyError: 0